In [ ]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd

from datetime import datetime


def get_historical_data(stk):
    df = yf.download(stk, period='max', interval="1wk").dropna()
    return df

def trade_parameters(df):
    
    df['abs_change']=abs(df.Close-df.Open)
    df['abs_change_roll2']=df.abs_change.rolling(2).sum()
    df['abs_change_roll3']=df.abs_change.rolling(3).sum()
    df['abs_change_roll4']=df.abs_change.rolling(4).sum()
     
    df['abs_change_rank_shift']=df.abs_change.rank(pct=True).shift(1)
    df['abs_change_rank_r2_shift']=df.abs_change_roll2.rank(pct=True).shift(1)
    df['abs_change_rank_r3_shift']=df.abs_change_roll3.rank(pct=True).shift(1)
    df['abs_change_rank_r4_shift']=df.abs_change_roll4.rank(pct=True).shift(1)


    df=df.iloc[4:,::]

    return df


def get_atm_option_data(symbol,expiry_date):
    ticker = yf.Ticker(symbol)
    last_price = ticker.history(period=expiry_date).Close[0]
    calls = ticker.option_chain(date=expiry_date).calls
    puts = ticker.option_chain(date=expiry_date).puts
    atm_call = calls[calls.inTheMoney == True].iloc[-1, :]
    atm_put = puts[puts.inTheMoney == False].iloc[-1, :]
    strike = atm_put.strike
    atm_call_last = (atm_call.bid + atm_call.ask) / 2
    atm_put_last = (atm_put.bid + atm_put.ask) / 2
    atm_call_IV = atm_call.impliedVolatility
    atm_put_IV = atm_put.impliedVolatility
    atm_data = [
        symbol,
        last_price,
        strike,
        atm_call_last,
        atm_put_last,
        atm_call_IV,
        atm_put_IV,
    ]
    return atm_data


expiry_date='2022-09-09'
arr = []

tickers=['MSOS','EWY','HYG','MJ','XRT','URA']
yields=[12.17,4.74,2.29,10.91,5.78,8.95]
for i,j in zip(tickers,yields):
    try:
        stock = i
        df = get_historical_data(stock)
        df = trade_parameters(df)
        df_opt=get_atm_option_data(stock,expiry_date)
        arr.append([i,
                    df.index[-1],
                    df.abs_change_rank_shift[-1],
                    df.abs_change_rank_r2_shift[-1],
                    df.abs_change_rank_r3_shift[-1],
                    df.abs_change_rank_r4_shift[-1],
                    df_opt[2],
                    df_opt[3],
                    df_opt[4],
                    j/100
                   ]
                  )
    except: 
        pass
finaldf = pd.DataFrame(
data=arr,
columns=[
    "stock",
    'last date',
    'abs_rank',
    'abs_rank_r2',
    'abs_rank_r3',
    'abs_rank_4k',
    'strike',
    'call',
    'put',
    'target_yield'

],
)
finaldf['current_yield']=(finaldf.call+finaldf.put)/finaldf.strike
finaldf['yield_diff']=(finaldf.current_yield-finaldf.target_yield)/finaldf.target_yield
print(finaldf)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
